In [13]:
""" 
this is a pytorch implementation of alexnet, trained and validated on 
CIFAR-10.

first time implementing a deep learning paper. 

using CIFAR-10 because ImageNet is too big to do locally

"""

### TODO ### 
# - import dependencies 
# - load the data
# - set up model architecture
# - define the forward pass 
# - write a training loop
# - train and validate model
# - final accuracy on test data

### add later ###
# - visualize loss 
# - visualize data
# - figure out how to get ImageNet

' \nthis is a pytorch implementation of alexnet, trained and validated on \nCIFAR-10.\n\nfirst time implementing a deep learning paper. \n\nusing CIFAR-10 because ImageNet is too big to do locally\n\n'

In [14]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment3/'
FOLDERNAME = 'cs231n'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# this downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd drive/My\ Drive/$FOLDERNAME/paper\ implementations/datasets
!bash get_datasets.sh
%cd /content

Mounted at /content/drive
/content/drive/My Drive/cs231n/paper implementations/datasets
/content


In [15]:
import torch
import torch.nn as nn

# to load the data
from torch.utils.data import DataLoader

# optimizer 
import torch.optim as optim

from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T 

import torch.nn.functional as F  # useful stateless functions

import numpy as np

In [16]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


In [17]:
# set up preprocessing transformations: 
# - resize to 256x256
# - take center crops 224x224
# - center with mean

# original paper resized image to 224x224, skip that becuase CIFAR-10
# has tiny images 


transform = T.Compose([
                T.ToTensor(),
                T.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
            ])

In [18]:
# load the data using a Dataset object and DataLoader wrapper
NUM_TRAIN = 49000

cifar10_train = dset.CIFAR10('./datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                         sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./datasets', train=True, download=True, 
                          transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                       sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [44]:
# set up model architecture

"""
model details: 
- 5 conv layers
- 3 FC layers
- relu nonlinearities
- local response normalization
- dropout
- momentum
- flatten between conv and fc layers
"""

class AlexNet(nn.Module): 
  def __init__(self, num_classes=10):
    super().__init__()

    # conv layers 
    self.conv1 = nn.Conv2d(3, 96, kernel_size=3, stride=1, padding=2)
    self.conv2 = nn.Conv2d(96, 256, kernel_size=3, stride=1, padding=2)
    self.conv3 = nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=2)
    self.conv4 = nn.Conv2d(384, 192, kernel_size=3, stride=1, padding=2)
    self.conv5 = nn.Conv2d(192, 2, kernel_size=3, stride=1, padding=2)

    # fully connected layers 
    self.fc1 = nn.Linear(392, 4096)
    self.fc2 = nn.Linear(4096, 4096)
    self.fc3 = nn.Linear(4096, num_classes)

    # overlapping pool where s < z 
    self.pool = nn.MaxPool2d(kernel_size=3, stride=2)

    # use F.relu for relu

    # local response norm
    # TODO: calc dims
    self.localresponsenorm = nn.LocalResponseNorm(size=5, alpha=1e-4, beta=0.75, k=2.0)

    # dropout regularization
    self.dropout = nn.Dropout()

    
  def forward(self, x): 
    # define connectivity
    x = self.conv1(x)
    x = self.localresponsenorm(x)
    x = F.relu(x)
    x = self.pool(x)

    x = self.conv2(x)
    x = self.localresponsenorm(x)
    x = F.relu(x)
    x = self.pool(x)

    x = self.conv3(x)
    x = F.relu(x)
    x = self.conv4(x)
    x = F.relu(x)
    x = self.conv5(x)
    x = F.relu(x)

    x = torch.flatten(x, 1)
    x = self.dropout(x)
    x = self.fc1(x)
    x = F.relu(x)

    x = self.dropout(x)
    x = self.fc2(x)
    x = F.relu(x)

    x = self.dropout(x)
    scores = self.fc3(x)

    return scores

In [45]:
def check_accuracy(loader, model): 
    if loader.dataset.train: 
        print('checking validation set')
    else: 
        print('checking test set')
    num_correct, num_samples = 0, 0
    model.eval()
    with torch.no_grad():
        for x, y in loader: 
            # put onto device (here using cpu)
            x = x.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=torch.long)
            
            # scores of val/test set
            scores = model(x)
            
            # preds are argmaxes of scores
            # torch.max returns tuple (values, indices)
            _, preds = scores.max(1)
            
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('%d / %d correct (%.2f)' % (num_correct, num_samples, 100*acc))
            

In [46]:
def train(model, optimizer, epochs=1):
    model = model.to(device=device)
    for e in range(epochs): 
        for t, (x, y) in enumerate(loader_train): 

            # puts model in train mode. call model.eval() for testing
            model.train() 

            # put onto device (here using cpu)
            x = x.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=torch.long)

            # forward pass 
            scores = model(x)

            # loss
            loss =  F.cross_entropy(scores, y)

            # zero the gradients
            optimizer.zero_grad()
            
            # backward pass
            loss.backward()

            # update parameters
            optimizer.step()

            # print progress 
            if t % 100 == 0: 
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy(loader_val, model)
                print()


In [49]:
model = AlexNet()
optimizer = optim.Adam(model.parameters())

In [51]:
# train the net
train(model, optimizer, epochs=10)

Iteration 0, loss = 1.5501
checking validation set
473 / 1000 correct (47.30)

Iteration 100, loss = 1.3336
checking validation set
486 / 1000 correct (48.60)

Iteration 200, loss = 1.5889
checking validation set
509 / 1000 correct (50.90)

Iteration 300, loss = 1.4998
checking validation set
548 / 1000 correct (54.80)

Iteration 400, loss = 1.3597
checking validation set
519 / 1000 correct (51.90)

Iteration 500, loss = 1.4498
checking validation set
549 / 1000 correct (54.90)

Iteration 600, loss = 1.2282
checking validation set
561 / 1000 correct (56.10)

Iteration 700, loss = 1.2364
checking validation set
530 / 1000 correct (53.00)

Iteration 0, loss = 1.2254
checking validation set
549 / 1000 correct (54.90)

Iteration 100, loss = 1.1313
checking validation set
544 / 1000 correct (54.40)

Iteration 200, loss = 1.3164
checking validation set
597 / 1000 correct (59.70)

Iteration 300, loss = 1.4482
checking validation set
591 / 1000 correct (59.10)

Iteration 400, loss = 1.0663
che

In [52]:
# for test set evaluation
best_model = model
check_accuracy(loader_test, best_model)

checking test set
7074 / 10000 correct (70.74)
